<a href="https://colab.research.google.com/github/Baek-Donghyeon/Bioinformatics-Algorithms/blob/main/Bioinformatics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Bioinformatics

In [154]:
import numpy
import random
import re
import sys
from collections import Counter
from collections import defaultdict
import itertools
from functools import reduce
import copy
sys.setrecursionlimit(3000)


def k_mer_of(text, k):
  for i in range(len(text) - k + 1):
    yield text[i:i+k]


def hamming_distance(pattern, text):
  # Minimum distance between pattern and all k_mers in text
  k = len(pattern)
  distance = k
  for k_mer in k_mer_of(text, k):
    distance = min(distance, sum(pattern[j] != k_mer[j] for j in range(k)))
  return distance


def pattern_to_number(pattern):
  symbol_to_number = {'A': 0, 'C': 1, 'G': 2, 'T': 3}
  if len(Pattern) == 0:
      return 0
  return 4*pattern_to_number(pattern[:-1]) + symbol_to_number[pattern[-1]]


def number_to_pattern(number, k):
  number_to_symbol = {0: 'A', 1: 'C', 2: 'G', 3: 'T'}
  try:
    if k == 1:
      return number_to_symbol(number)
    return number_to_pattern(number // 4, k-1) + number_to_symbol[number % 4]
  except:
    raise ValueError()


def count_pattern(text, pattern):
  return sum(k_mer == pattern for k_mer in k_mer_of(text, len(pattern)))


def find_frequent_k_mer(text, k, distance = 0, reverse_complement = False):
  ''' k   length of the k_mer '''
  frequency_dict = dict(frequency_dict(text, k, distance, reverse_complement))
  max = max(frequency_dict, key = lambda x : frequency_dict[x])
  return list(filter(lambda x : x[1] == max, frequency_dict.items())) 


def frequency_dict(text, k, distance = 0, reverse_complement = False):
  ''' k   length of the k_mer '''
  if reverse_complement:
    return frequency_dict(reverse_complement(text), k, distance) + frequency_dict(text, k, distance)
  else:
    return Counter([neighbor for k_mer in k_mer_of(text, k) for neighbor in neighbor(k_mer, distance)])


def reverse_complement(text):
  compliment_dict = {'A': 'T', 'T': 'A', 'G': 'C', 'C': 'G'}
  return ''.join(reversed([compliment_dict[base] for base in text]))


def find_pattern_occurence(text, pattern, distance = 0):
  # hamming_distance(text, pattern) ≤ d
  k = len(pattern)
  return [i for i in range(len(text) - k + 1) if hamming_distance(text[i:i+k], pattern) <= distance]


def find_clump(text, k, L, t):
  '''
  k   length of the k_mer
  L   length of an interval
  t   min ocurrences of a pattern in the interval
  '''
  position_dict = defaultdict(list)
  frequency_dict = frequency_dict(text, k)
  for pattern, frequency in frequency_dict.items():
    if frequency >= t:
      position_dict[pattern] = find_occurrences_of_pattern(text, pattern)
  for pattern, position in position_dict.items():
    for i in range(len(position) - t + 1):
      if position[i+t-1] - position[i] <= L - k + 1:
        yield pattern


def find_min_skew_position(text):
  # skew : abs(#'G'-#'C')
  skew = [0]
  for base in text:
    if base == 'C': skew.append(skew[-1]-1)
    elif base == 'G': skew.append(skew[-1]+1)
    else: skew.append(skew[-1])
  min = min(skew)
  return list(filter(lambda x : x == min, skew)) # return minimum skew


def neighbor(pattern, distance):
  length = len(pattern)
  mismatch_base = itertools.product("ACTG", repeat = distance) # all possible 'distance' length letter strings
  for positions in itertools.combinations(iter(range(length)), distance):
    for base in mismatch_base:
      neighbor = copy.deepcopy(pattern)
      for i in distance:
        neighbor[position[i]] = base[i]
        yield neighbor
      

def enumerate_motif(strings, k, distance):
  # Find all (k, distance)-motifs in a collection of strings.
  neighbor_list = []
  for text in strings:
    neighbor_list.append(set(neighbor(text, k, distance)))
  motifs = neighbor_list[0]
  for neighbor in neighbor_list:
    motifs = motifs & neighbor
  return list(motifs)


def find_median_string(strings, k, distance = 0):
  # A median string for Dna minimizes d(Pattern, Dna) over all k-mers Pattern.
  motifs = enumerate_motif(strings, k, distance)
  if not motifs:
    find_median_string(strings, k, distance + 1)
  else:
    motifs_dict = defaultdict(int)
    for pattern in motifs:
      motifs_dict[pattern] = sum(hamming_distance(pattern, text) for text in strings)
    return min(motifs_dict, key = lambda x : motifs_dict[x]) # return key that minimizes the value


def find_profile_probable_k_mer(text, k, profile):
  # A k-mer that was most likely to have been generated by Profile among all k-mers in Text.
  score = 0
  for k_mer in k_mer_of(text, k):
    score = max(score, score(profile, [k_mer]))
  return score


def score(profile, motif):
  return sum(reduce(lambda x, y: x*y, list(profile[pattern_to_number(k_mer[i])][i]
              for i in range(k))) for k_mer in motif)


def form_profile(motif):
  k = len(motif[0])
  profile = numpy.full((4,k), 1) # pseudocount
  for i in range(k):
    for k_mer in motif:
      Profile[pattern_to_number(k_mer[i])][i] += 1
  return profile/(len(motif)+4)


def greedy_motif_search(strings, k):
  best_motif = [text[:k] for text in strings]
  for i in range(len(strings[0]) - k + 1):
    motif = [strings[0][i:i+k]]
    for j in range(1,len(strings)):
      profile = form_profile(motif)
      motif.append(find_profile_probable_k_mer(strings[j], k, motif))
    if score(profile, motif) < score(profile, best_motif):
      best_motif = motif
  return best_motif


# def RandomizedMotifSearch(Dna, k, t):
#   w = len(Dna[0])
#   # from Symbol to Number representation
#   NumDna = [[SymbolToNumber(Symbol) for Symbol in list(line)] for line in Dna]
#   Motifs = []
#   for i in range(t):
#     j = random.randrange(w-k+1)
#     Motifs.append(NumDna[i][j:j+k])
#   BestMotifs = Motifs
#   for x in range(1000):
#     while True:
#       Profile = ProfileFormation(Motifs, k)
#       NewMotifs = []
#       for Text in NumDna:
#         NewMotifs.append(MostProbableKmer(Text, k, Profile))
#       Motifs = NewMotifs
#       if Score(Motifs, Profile) > Score(BestMotifs, Profile):
#         BestMotifs = Motifs
#       else:
#         break
#     Motifs = []
#     for i in range(t):
#       j = random.randrange(w-k+1)
#       Motifs.append(NumDna[i][j:j+k])
#   # from Number to Symbol representation
#   return [''.join([NumberToSymbol(Number) for Number in line]) for line in BestMotifs]


# def GibbsSampler(Dna, k, t, N):
#   w = len(Dna[0])
#   # from Symbol to Number representation
#   NumDna = [[SymbolToNumber(Symbol) for Symbol in list(line)] for line in Dna]
#   Motifs = []
#   for i in range(t):
#     j = random.randrange(w-k+1)
#     Motifs.append(NumDna[i][j:j+k])
#   BestMotifs = Motifs
#   for j in range(N):
#     i = random.randrange(t)
#     Motifs.pop(i)
#     Profile = ProfileFormation(Motifs, k)
#     #Profile-randomly generated k-mer in a string Text.
#     Motifs.insert(i, MostProbableKmer(NumDna[i], k, Profile))
#     if Score(Motifs, Profile) > Score(BestMotifs, Profile):
#       BestMotifs = Motifs
#   return [''.join([NumberToSymbol(Number) for Number in line]) for line in BestMotifs]


def find_string(strings, d = None):
  first = strings[0]
  if d != None: # Paired-Reads
    half = len(first)//2
    return find_string([front[:half] for front in strings]) + find_string([rear[half:] for rear in strings[-half-d:]])
  return first[:-1] + ''.join([text[-1] for text in strings])


def de_brujin_graph(strings):
  graph = defaultdict(list)
  if len(strings[0]) == 2: # Paired-Reads
    for f,r in strings: # ex) GAGA|TTGA, f = GAGA, r = TTGA
      graph[f[:-1]+r[:-1]].append(f[1:]+r[1:])         
  else:
    for k_mer in strings:
      graph[k_mer[:-1]].append(k_mer[1:])
  return graph


def eulerian(graph):

  # determine whether it is an eulerian path or cycle
  graph = copy.deepcopy(graph)
  in_degree = Counter(list(itertools.chain(*graph.values())))
  out_degree = Counter([k for k,v in graph.items() for __ in range(len(v))])
  odd_degree = (out_degree - in_degree).keys()
  if odd_degree:
    stack = list(odd_degree) # start from odd_degree / eulerian path
  else:
    stack = [list(graph.keys())[0]] # start from arbitrary node / eulerian cycle

  # DFS
  visit = []
  while stack:
    try:
      stack.append(graph[stack[-1]].pop())
    except: # backtrack
      visit.append(stack.pop())
  visit.reverse()
  return visit


def reconstruct_string(strings, d = None):
  # d = # of bases b/w front and rear ends
  return find_string(eulerian(de_brujin_graph(strings)),d)


def find_k_universal_circular_string(k):
  # all possible k_mers
  # [:-k+1] : for circular_string
  return reconstruct_string(list(map(''.join, product('01', repeat=k))))[:-k+1]


def generate_contig(strings):
  graph = de_brujin_graph(strings)
  in_degree = Counter(list(itertools.chain(*graph.values())))
  out_degree = Counter([k for k,v in graph.items() for __ in range(len(v))])
  collection = []
  branch = set(node for node in graph.keys() if in_degree[node] != 1 or out_degree[node] != 1)
  branch.update(set(node for values in graph.values() for node in values if in_degree[node] != 1 or out_degree[node] != 1))
  for node in branch:
    while graph[node]:
      contig = [node, graph[node].pop()]
      while contig[-1] not in branch:
        contig.append(graph[contig[-1]].pop())
      collection.append(find_string(contig))
  return collection


f = open('/content/drive/My Drive/Colab Notebooks/input.txt', 'r')
strings = []
for line in f.readlines():
  strings.append(line.strip())
for contig in sorted(generate_contig(strings)):
  print(contig)
f.close()

  # def ContigGeneration(PathGraph):
  # #Generate the contigs from a collection of reads (with imperfect coverage).
  # #Non-branching if in(v) = out(v) = 1 for each intermediate node v of this path,
  #   Graph = []
  #   AdjDict = {}
  #   DegreeDict = {} # Degree : # of nodes [In, Out]
  #   for Nodes in PathGraph:
  #     for Node in Nodes:
  #       AdjDict.setdefault(Node)
  #       DegreeDict.setdefault(Node, [0,0])
  #   for Nodes in PathGraph:
  #     AdjDict[Nodes[0]] = Nodes[1:]
  #     DegreeDict[Nodes[0]][1] = len(Nodes[1:])

  #   for EndNodes in AdjDict.values():
  #     if EndNodes:
  #       for EndNode in EndNodes:
  #         DegreeDict[EndNode][0] = (DegreeDict[EndNode][0]+1)

  #   def Search(key, contig):
  #     value = AdjDict[key].pop()
  #     contig.append(value)
  #     if DegreeDict[value] == [1,1] and AdjDict[value]:
  #       return Search(value, contig)
  #     else: # Finish Search if value is a branch point or has no out
  #       return Reconstruct(contig)
    
  #   Contigs = []
  #   IsNotEmpty = True
  #   while IsNotEmpty:
  #     IsNotEmpty = False
  #     for key in AdjDict.keys():
  #       if AdjDict[key] and not DegreeDict[key] == [1,1]:
  #       # key is a branch point
  #         Contigs.append(Search(key, [key]))
  #         IsNotEmpty = True

  #   return Contigs


  # def ProteinTranslation(Pattern):
  #   Protein = []
  #   for i in range(len(Pattern)//3):
  #     Codon = Pattern[3*i:3*i+3]
  #     if Codon in ['UAA','UAG','UGA']:
  #       break
  #     Protein.append(GeneticCode[Codon])
  #   return ''.join(Protein)


  # def DnaToRna(Dna):
  #   return re.sub('T','U',Dna)


  # def PeptideEncoding(Text, Peptide):
  # #Find substrings of a genome encoding a given amino acid sequence.
  #   Substrings = []
  #   PeptideLen = len(Peptide)
  #   for i in range(len(Text)-3*PeptideLen+1):
  #     substring = Text[i:i+3*PeptideLen]
  #     if ProteinTranslation(DnaToRna(substring)) == Peptide or ProteinTranslation(DnaToRna(ReverseCompliment(substring))) == Peptide:
  #       Substrings.append(substring)
  #   return Substrings


  # def Cyclospectrum(Peptide):
  # #Generate the theoretical spectrum of a cyclic peptide.
  # #The theoretical spectrum of a cyclic peptide Peptide is the collection of all of the masses of its subpeptides
  #   PeptideLen = len(Peptide)
  #   TheoreticalSpectrum = [0,Mass(Peptide)]

  #   Peptide += Peptide
  #   for i in range(1,PeptideLen):
  #     for j in range(PeptideLen):
  #       TheoreticalSpectrum.append(Mass(Peptide[j:j+i]))
    
  #   return sorted(TheoreticalSpectrum)


  # def Mass(Peptide):
  #   Mass = 0
  #   for AminoAcid in Peptide:
  #     Mass += IntegerMass[AminoAcid]
  #   return Mass


  # def BFCyclopeptideSequencing(Mass):
  # #Compute the number of peptides of given total mass.
  #   if Mass in ReducedMass.values():
  #     global cnt
  #     cnt += 1
  #   elif Mass > 57:
  #     for AminoAcid in ReducedMass.values():
  #       if Mass > AminoAcid:
  #         BFCyclopeptideSequencing(Mass-AminoAcid)


  # def CyclopeptideSequencing(Spectrum):
    
  #   def Expand(CandidatePeptides):
  #     Expand = []
  #     for Peptide in CandidatePeptides:
  #       for AminoAcid in ReducedMass.keys():
  #         Expand.append(Peptide+AminoAcid)
  #     return Expand
  #   CandidatePeptides = list(ReducedMass.keys())
  #   FinalPeptides = []
  #   while True:
  #     CopyCandidatePeptides = CandidatePeptides.copy()
  #     for Peptide in CopyCandidatePeptides:
  #       Peptidespectrum = Cyclospectrum(Peptide)
  #       if Peptidespectrum == Spectrum:
  #         FinalPeptides.append(Peptide)
  #         CandidatePeptides.remove(Peptide)
  #       else:
  #         for Fragment in Peptidespectrum:
  #           if Fragment not in Spectrum:
  #             CandidatePeptides.remove(Peptide)
  #             break
  #     if not CandidatePeptides:
  #       break
  #     print(CandidatePeptides)
  #     CandidatePeptides = Expand(CandidatePeptides)
  #   print(FinalPeptides)
  #   return FinalPeptides


  # GeneticCode = {'AAA':'K','AAC':'N','AAG':'K','AAU':'N','ACA':'T','ACC':'T','ACG':'T','ACU':'T',
  #               'AGA':'R','AGC':'S','AGG':'R','AGU':'S','AUA':'I','AUC':'I','AUG':'M','AUU':'I',
  #               'CAA':'Q','CAC':'H','CAG':'Q','CAU':'H','CCA':'P','CCC':'P','CCG':'P','CCU':'P',
  #               'CGA':'R','CGC':'R','CGG':'R','CGU':'R','CUA':'L','CUC':'L','CUG':'L','CUU':'L',
  #               'GAA':'E','GAC':'D','GAG':'E','GAU':'D','GCA':'A','GCC':'A','GCG':'A','GCU':'A',
  #               'GGA':'G','GGC':'G','GGG':'G','GGU':'G','GUA':'V','GUC':'V','GUG':'V','GUU':'V',
  #               'UAA':'*','UAC':'Y','UAG':'*','UAU':'Y','UCA':'S','UCC':'S','UCG':'S','UCU':'S',
  #               'UGA':'*','UGC':'C','UGG':'W','UGU':'C','UUA':'L','UUC':'F','UUG':'L','UUU':'F'}

  # IntegerMass = {'G':57,'A':71,'S':87,'P':97,'V':99,'T':101,'C':103,'I':113,'L':113,'N':114,
  #               'D':115,'K':128,'Q':128,'E':129,'M':131,'H':137,'F':147,'R':156,'Y':163,'W':186}

  # ReducedMass = {'G':57,'A':71,'S':87,'P':97,'V':99,'T':101,'C':103,'I':113,'N':114,
  #               'D':115,'K':128,'E':129,'M':131,'H':137,'F':147,'R':156,'Y':163,'W':186}
  #Spectrum = list(map(int,f.readline().split(' ')))
  # print(CyclopeptideSequencing(Spectrum))




AAAAACTTGAAGGCTTAT
AAAAACTTGAAGGCTTAT
AAAACTTGAAGGCTTATG
AAAACTTGAAGGCTTATG
AAAAGTGGGCGCGCGCCG
AAAAGTGGGCGCGCGCCG
AAACTTGAAGGCTTATGGCCCCAGTGCGTA
AAAGTGGGCGCGCGCCGC
AAAGTGGGCGCGCGCCGC
AACATTAATAGCTGCTCTAAGCCCTCCGCA
AACGAAAACCTAAAGGCT
AACGAAAACCTAAAGGCT
AACGGCCCCGGGCGCCCTCATCGACCTGTTA
AACTCTCCACCCGGGCTG
AACTCTCCACCCGGGCTG
AAGAAAACGTTGATATGTCTCTGAGCATGAT
AAGAATTGGCCAGCGAGA
AAGAATTGGCCAGCGAGA
AAGCGAAATCTTCTCTAA
AAGCGAAATCTTCTCTAA
AAGCGTCAGCGAGATGCGGAGGCCAACGACAGG
AAGTGGGCGCGCGCCGCAAGAAAACGTTGATA
AAGTGGTTGAAAACAAGTCGAAGTGTTATGGATAG
AATACACCCTGCAGAGTATTACTTTAGTCCCTTA
AATCGTTCAGGCTACGAA
AATCGTTCAGGCTACGAA
AATCTGCGATGGATAGAGGGGATGCCCTTCGGG
ACCAGGTACGGATATCTG
ACCAGGTACGGATATCTG
ACCCCGCTAGTGGACCTG
ACCCCGCTAGTGGACCTG
ACCCTGGTGCAGAAGAGG
ACCCTGGTGCAGAAGAGG
ACCGTAGCGCGCGCGGCA
ACCGTAGCGCGCGCGGCA
ACGAAAACCTAAAGGCTGAGTGTCTACTTGGGTT
ACGCGGAGCGTCTGTGGT
ACGCGGAGCGTCTGTGGT
ACGTGTTTCATACCCCAT
ACGTGTTTCATACCCCAT
ACGTTTCCCCGAGTCTGAAGTGGTTGAAAACAA
ACTATGTGTCGGACCGGA
ACTATGTGTCGGACCGGA
ACTCTCCACCCGGGCTGTAACATTA

In [73]:
+for front, rear in [[1,2],[2,3]]:
  print(front, rear)

1 2
2 3


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
